# Experiment 6: MLP network with Bag of Words features

Our first deep learning experiment following the proof of concept in the parent directory, this notebook will experiment with the most basic form of neural network, which is simply a multi-layer feedforward network (multilayer perceptron) with all nodes connected to every other node. However, in this one we will use a much bigger dataset, because the previous notebook is deceptive(lol) in its accuracy figures because it's such a small, similar dataset.

First, let's import and split our data. We'll use 50,000 reviews.

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from keras.preprocessing import text
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from exp4_data_feature_extraction import get_balanced_dataset
from scripts import training_helpers as th

reviews_set, fake_reviews, genuine_reviews, unused_genuine_reviews = get_balanced_dataset()
reviews = reviews_set[:50000]
X = [x.review_content for x in reviews]
y = np.array([x.label for x in reviews])

Using TensorFlow backend.


First, lets limit the number of words we use from our reviews, to filter out some of the nonsense. 10,000 is as good a number as any.

In [2]:
NUM_WORDS = 10000
tokenizer = text.Tokenizer(num_words=NUM_WORDS)
tokenizer.fit_on_texts(X)

Now, let's define a function that takes a bunch of reviews and returns them as word count vectors of size 10,000.

In [3]:
def tokenize(data):
    return tokenizer.texts_to_matrix(data, mode='count')

X = np.array(tokenize(X))

Let's take a look at a review to make sure it looks ok.

In [4]:
print(X[0])

[0. 0. 0. ... 0. 0. 0.]


## Initial fully connected FF network.

Now let's train and validate a model to see what our accuracies look like. 
This time, to help prevent overfitting, we're going to use k-fold cross validation, with k=10, to make sure our model isn't biased to a particular chunk of the dataset.
We wont use any regularization methods this time around so we can see how they affect it when we add them in to our layers later on.

In [5]:
from keras.callbacks import EarlyStopping
from keras import regularizers

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cvscores = []
for train, test in kfold.split(X, y):
    model = keras.Sequential([
    keras.layers.Dense(16, activation=tf.nn.relu, input_shape=(NUM_WORDS,)),
    keras.layers.Dense(16, activation=tf.nn.relu,),
    keras.layers.Dense(1, activation=tf.nn.sigmoid)
    ])
    model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
    model.fit(X[train], y[train], epochs=10, batch_size=2048, validation_split=0.3, verbose=1)
    scores = model.evaluate(X[test], y[test], verbose=2)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

Train on 31499 samples, validate on 13500 samples
Epoch 1/10
 - 15s - loss: 0.6676 - acc: 0.5913 - val_loss: 0.6390 - val_acc: 0.6581
Epoch 2/10
 - 7s - loss: 0.6079 - acc: 0.6834 - val_loss: 0.6196 - val_acc: 0.6719
Epoch 3/10
 - 6s - loss: 0.5725 - acc: 0.7050 - val_loss: 0.6158 - val_acc: 0.6741
Epoch 4/10
 - 7s - loss: 0.5457 - acc: 0.7224 - val_loss: 0.6182 - val_acc: 0.6729
Epoch 5/10
 - 7s - loss: 0.5223 - acc: 0.7393 - val_loss: 0.6282 - val_acc: 0.6690
Epoch 6/10
 - 7s - loss: 0.5005 - acc: 0.7534 - val_loss: 0.6374 - val_acc: 0.6656
Epoch 7/10
 - 7s - loss: 0.4796 - acc: 0.7675 - val_loss: 0.6505 - val_acc: 0.6603
Epoch 8/10
 - 6s - loss: 0.4597 - acc: 0.7818 - val_loss: 0.6662 - val_acc: 0.6584
Epoch 9/10
 - 6s - loss: 0.4428 - acc: 0.7924 - val_loss: 0.6807 - val_acc: 0.6556
Epoch 10/10
 - 7s - loss: 0.4232 - acc: 0.8061 - val_loss: 0.7014 - val_acc: 0.6526
acc: 65.49%
Train on 31499 samples, validate on 13500 samples
Epoch 1/10
 - 15s - loss: 0.6731 - acc: 0.5609 - val_los

 - 7s - loss: 0.6293 - acc: 0.6724 - val_loss: 0.6247 - val_acc: 0.6648
Epoch 3/10
 - 5s - loss: 0.5866 - acc: 0.6990 - val_loss: 0.6156 - val_acc: 0.6697
Epoch 4/10
 - 5s - loss: 0.5570 - acc: 0.7197 - val_loss: 0.6154 - val_acc: 0.6685
Epoch 5/10
 - 6s - loss: 0.5304 - acc: 0.7378 - val_loss: 0.6211 - val_acc: 0.6682
Epoch 6/10
 - 6s - loss: 0.5052 - acc: 0.7539 - val_loss: 0.6304 - val_acc: 0.6646
Epoch 7/10
 - 7s - loss: 0.4791 - acc: 0.7718 - val_loss: 0.6457 - val_acc: 0.6608
Epoch 8/10
 - 7s - loss: 0.4525 - acc: 0.7890 - val_loss: 0.6603 - val_acc: 0.6577
Epoch 9/10
 - 5s - loss: 0.4252 - acc: 0.8079 - val_loss: 0.6836 - val_acc: 0.6544
Epoch 10/10
 - 4s - loss: 0.3997 - acc: 0.8249 - val_loss: 0.7010 - val_acc: 0.6536
acc: 63.21%
65.06% (+/- 0.86%)


Awesome. With only one hidden layer, 8 nodes, simple word count embeddings as features and no dropout or regularization, we get 65% accuracy. Not as good as our POC, but with over 30x the data, definitley a good first step.

We can see above that validation loss begins to decrease, but then starts to increase, while training loss and accuracy continues to increase. 
This indicates that the model is overfitting. It continues to get better and better at fitting the data that it sees (training data) while getting worse and worse at fitting the data that it does not see (validation data).


## Tackling overfitting

There's multiple methods of tackling overfitting. We can look at:

- Regularization functions (L1, L2)
- Dropout layers (Sets random fraction of inputs to 0 during training)
- Early stopping callbacks, to stop training when validation loss is not improving

We'll try all of these, as well as a couple of different architectures.

First, let's define our early stopping callback function, so we don't waste time training with diminishing returns. This will stop training when validation loss doesn't decrease after 3 epochs.


In [14]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

Now let's train our previous model, with early stopping and regularization.

There are three types of layer regularization in Keras: kernel, bias, and activity.

Kernel: this applies to actual weights of the layer, in Dense it is the W of Wx+b.

Bias: this is the bias vector of the weights, so you can apply a different regulariser for it, the b in Wx+b.

Activity: is applied to the output vector, the y in y = f(Wx + b).

To use regularization to prevent overfitting, we want to apply it to the kernel weights.

Let's start with adding L2 regularization with early stopping.

In [15]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cvscores = []
for train, test in kfold.split(X, y):
    model = keras.Sequential([
    keras.layers.Dense(16, activation=tf.nn.relu, input_shape=(NUM_WORDS,), kernel_regularizer=regularizers.l2(0.01)),
    keras.layers.Dense(16, activation=tf.nn.relu, kernel_regularizer=regularizers.l2(0.01)),
    keras.layers.Dense(1, activation=tf.nn.sigmoid)
    ])
    model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
    model.fit(X[train], y[train], epochs=30, batch_size=1024, validation_split=0.3, verbose=0, callbacks=[early_stop])
    scores = model.evaluate(X[test], y[test], verbose=2)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

Train on 27999 samples, validate on 12000 samples
Epoch 1/30
27999/27999 [==============================] - 12s 433us/step - loss: 0.9126 - acc: 0.5849 - val_loss: 0.7914 - val_acc: 0.6602
Epoch 2/30
27999/27999 [==============================] - 3s 109us/step - loss: 0.7609 - acc: 0.6698 - val_loss: 0.7547 - val_acc: 0.6668
Epoch 3/30
27999/27999 [==============================] - 3s 102us/step - loss: 0.7272 - acc: 0.6794 - val_loss: 0.7363 - val_acc: 0.6658
Epoch 4/30
27999/27999 [==============================] - 3s 98us/step - loss: 0.7073 - acc: 0.6826 - val_loss: 0.7212 - val_acc: 0.6683
Epoch 5/30
27999/27999 [==============================] - 3s 113us/step - loss: 0.6915 - acc: 0.6847 - val_loss: 0.7083 - val_acc: 0.6652
Epoch 6/30
27999/27999 [==============================] - 3s 106us/step - loss: 0.6801 - acc: 0.6833 - val_loss: 0.6997 - val_acc: 0.6643
Epoch 7/30
27999/27999 [==============================] - 3s 102us/step - loss: 0.6691 - acc: 0.6863 - val_loss: 0.6903 - 

Epoch 6/30
28000/28000 [==============================] - 6s 217us/step - loss: 0.6786 - acc: 0.6944 - val_loss: 0.7041 - val_acc: 0.6679
Epoch 7/30
28000/28000 [==============================] - 6s 220us/step - loss: 0.6687 - acc: 0.6948 - val_loss: 0.6965 - val_acc: 0.6673
Epoch 8/30
28000/28000 [==============================] - 5s 180us/step - loss: 0.6583 - acc: 0.6971 - val_loss: 0.6910 - val_acc: 0.6665
Epoch 9/30
28000/28000 [==============================] - 4s 153us/step - loss: 0.6528 - acc: 0.6972 - val_loss: 0.6905 - val_acc: 0.6582
Epoch 10/30
28000/28000 [==============================] - 4s 143us/step - loss: 0.6462 - acc: 0.6986 - val_loss: 0.6789 - val_acc: 0.6673
Epoch 11/30
28000/28000 [==============================] - 5s 164us/step - loss: 0.6394 - acc: 0.6982 - val_loss: 0.6745 - val_acc: 0.6670
Epoch 12/30
28000/28000 [==============================] - 5s 163us/step - loss: 0.6354 - acc: 0.6985 - val_loss: 0.6711 - val_acc: 0.6672
Epoch 13/30
28000/28000 [======

Already a slight improvement, and the models validation loss decreases far better. Let's add a dropout layer to the mix before adding more dense layers.

In [19]:
cvscores = []
for train, test in kfold.split(X, y):
    model = keras.Sequential([
    keras.layers.Dense(16, activation=tf.nn.relu, input_shape=(NUM_WORDS,), kernel_regularizer=regularizers.l2(0.01)),
    keras.layers.Dropout(0.25),
    keras.layers.Dense(16, activation=tf.nn.relu, kernel_regularizer=regularizers.l2(0.01)),
    keras.layers.Dense(1, activation=tf.nn.sigmoid)
    ])
    model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
    model.fit(X[train], y[train], epochs=30, batch_size=1024, validation_split=0.3, verbose=1, callbacks=[early_stop])
    scores = model.evaluate(X[test], y[test], verbose=2)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

Train on 27999 samples, validate on 12000 samples
Epoch 1/30
27999/27999 [==============================] - 15s 546us/step - loss: 0.9434 - acc: 0.5746 - val_loss: 0.8203 - val_acc: 0.6516
Epoch 2/30
27999/27999 [==============================] - 5s 175us/step - loss: 0.7986 - acc: 0.6503 - val_loss: 0.7770 - val_acc: 0.6642
Epoch 3/30
27999/27999 [==============================] - 4s 147us/step - loss: 0.7621 - acc: 0.6633 - val_loss: 0.7518 - val_acc: 0.6644
Epoch 4/30
27999/27999 [==============================] - 4s 137us/step - loss: 0.7363 - acc: 0.6669 - val_loss: 0.7339 - val_acc: 0.6656
Epoch 5/30
27999/27999 [==============================] - 3s 112us/step - loss: 0.7166 - acc: 0.6706 - val_loss: 0.7178 - val_acc: 0.6625
Epoch 6/30
27999/27999 [==============================] - 3s 111us/step - loss: 0.7008 - acc: 0.6743 - val_loss: 0.7043 - val_acc: 0.6682
Epoch 7/30
27999/27999 [==============================] - 3s 113us/step - loss: 0.6869 - acc: 0.6769 - val_loss: 0.6950 -

Epoch 16/30
28000/28000 [==============================] - 3s 119us/step - loss: 0.6298 - acc: 0.6982 - val_loss: 0.6617 - val_acc: 0.6688
Epoch 17/30
28000/28000 [==============================] - 4s 128us/step - loss: 0.6275 - acc: 0.6993 - val_loss: 0.6613 - val_acc: 0.6716
Epoch 18/30
28000/28000 [==============================] - 5s 181us/step - loss: 0.6255 - acc: 0.7007 - val_loss: 0.6625 - val_acc: 0.6687
acc: 66.52%
Train on 28000 samples, validate on 12001 samples
Epoch 1/30
28000/28000 [==============================] - 16s 580us/step - loss: 0.9132 - acc: 0.5877 - val_loss: 0.7930 - val_acc: 0.6513
Epoch 2/30
28000/28000 [==============================] - 4s 132us/step - loss: 0.7706 - acc: 0.6584 - val_loss: 0.7557 - val_acc: 0.6613
Epoch 3/30
28000/28000 [==============================] - 3s 117us/step - loss: 0.7391 - acc: 0.6675 - val_loss: 0.7341 - val_acc: 0.6587
Epoch 4/30
28000/28000 [==============================] - 3s 121us/step - loss: 0.7181 - acc: 0.6728 - val

Not much of a difference, but with only one hidden layer that's not overly surpising. Let's increase the number of nodes in the first layer to 32, add another hidden layer of size 16 with another dropout layer before it, and see what happens. Also, let's reduce cross validation to 3 folds to save some time.

In [25]:
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
cvscores = []
for train, test in kfold.split(X, y):
    model = keras.Sequential([
    keras.layers.Dense(16, activation=tf.nn.relu, input_shape=(NUM_WORDS,), kernel_regularizer=regularizers.l2(0.01)),
    keras.layers.Dropout(0.25),
    keras.layers.Dense(8, activation=tf.nn.relu, kernel_regularizer=regularizers.l2(0.01)),
    keras.layers.Dense(1, activation=tf.nn.sigmoid)
    ])
    model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
    model.fit(X[train], y[train], epochs=30, batch_size=512, validation_split=0.3, verbose=1, callbacks=[early_stop])
    scores = model.evaluate(X[test], y[test], verbose=2)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

Train on 23332 samples, validate on 10000 samples
Epoch 1/30
23332/23332 [==============================] - 15s 639us/step - loss: 0.9114 - acc: 0.5734 - val_loss: 0.8184 - val_acc: 0.6517
Epoch 2/30
23332/23332 [==============================] - 3s 116us/step - loss: 0.8002 - acc: 0.6284 - val_loss: 0.7734 - val_acc: 0.6593
Epoch 3/30
23332/23332 [==============================] - 3s 112us/step - loss: 0.7658 - acc: 0.6448 - val_loss: 0.7474 - val_acc: 0.6592
Epoch 4/30
23332/23332 [==============================] - 3s 117us/step - loss: 0.7409 - acc: 0.6511 - val_loss: 0.7278 - val_acc: 0.6610
Epoch 5/30
23332/23332 [==============================] - 3s 115us/step - loss: 0.7199 - acc: 0.6579 - val_loss: 0.7137 - val_acc: 0.6533
Epoch 6/30
23332/23332 [==============================] - 3s 114us/step - loss: 0.7055 - acc: 0.6606 - val_loss: 0.7005 - val_acc: 0.6607
Epoch 7/30
23332/23332 [==============================] - 3s 116us/step - loss: 0.6931 - acc: 0.6625 - val_loss: 0.6923 -

Epoch 18/30
23333/23333 [==============================] - 3s 114us/step - loss: 0.6458 - acc: 0.6924 - val_loss: 0.6661 - val_acc: 0.6668
acc: 66.60%
66.37% (+/- 0.22%)
